In [1]:
import json
from py2neo import Graph
import pandas as pd
import subprocess
import re

In [ ]:
chunks = pd.read_json("arxiv-metadata-oai-snapshot.json", lines=True, chunksize = 300000)
i = 0
for c in chunks:
    c.to_csv(f"neo4j/import/data_{i}.csv", index=False)
    i += 1

In [48]:
data = pd.read_csv('neo4j/import/data_1.csv')

In [49]:
# remove punctuation!
data = data[['id', 'authors', 'title']]
data['authors'] = data['authors'].apply(lambda x: re.split(', | and ', x))
data = data.explode('authors')
data['authors'] = data['authors'].fillna('')
data['authors'] = data['authors'].apply(lambda x: x.replace('"', ''))
data['authors'] = data['authors'].apply(lambda x: x.replace("'", ''))
data['authors'] = data['authors'].apply(lambda x: x.replace('\\',''))
data['title'] = data['title'].fillna('')
data['title'] = data['title'].apply(lambda x: x.replace('"', ''))

In [50]:
data['author_id'] = data['authors'].apply(lambda x: x.split(' ')[-1].lower())
data['author_label'] = 'Author'
data['paper_label'] = 'Paper'
data['relationship_type'] = 'WROTE'
data = data[data['author_id'] != '']

In [65]:
data[['id', 'title', 'paper_label']].drop_duplicates().to_csv('neo4j/import/papers1.csv', index=False, header=False)
data[~data['author_id'].isin(pd.read_csv('neo4j/import/authors.csv', header=None)[0])][['author_id', 'authors', 'author_label']].drop_duplicates('author_id').to_csv('neo4j/import/authors1.csv', index=False, header=False)
data[['author_id', 'id', 'relationship_type']].drop_duplicates().to_csv('neo4j/import/relations1.csv', index=False, header=False)

paperId:ID,title,:LABEL

personId:ID,name,:LABEL

:START_ID,:END_ID,:TYPE

In [34]:
try:
    graph = Graph("bolt://localhost:7687")
except:
    print("Error Connection to Neo4j DB!!")

Error Connection to Neo4j DB!!


In [ ]:
graph.delete_all()